# Purpose: Mine Tweets from trending topics, clean and find most used words with in those tweets, output to DF

## Some Boring Twitter Rules

Twitter says they will rate limit your requests:

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rulez":

![](https://snag.gy/yJ6vIH.jpg)


In [216]:
import twitter, re, datetime, pandas as pd
import tweepy
import datetime
import re
import HTMLParser
import numpy as np
from nltk.corpus import stopwords # Import the stop word list

In [239]:
twitter.api 

<module 'twitter.api' from '/Users/Stav/anaconda2/lib/python2.7/site-packages/twitter/api.pyc'>

In [288]:
twitter_keys = {
    'consumer_key':        'dhZbJCvhl79hmz6eGpsy6WRfY',
    'consumer_secret':     'TaBDp9pXOlQcxbhELuyzDFilEz5onut1hDtuOAPOYmQu15Veby',
    'access_token_key':    '729878120338063360-RuyFwFOyq9rsRk0X8FMhkbnBxeasaL0',
    'access_token_secret': 'TRuE65px8xUj76ayD2ffiv3lhabTTFHBHp1R6fa92k8oU'
}

auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth)

In [241]:
for tweet in tweepy.Cursor(api.search, q="#BaeIn3Words", lang="tr").items():
    print tweet

In [314]:
trending_list = []
trends = api.trends_place(2487956)
for i in trends:
    for trend in i['trends']:
        if trend['tweet_volume'] > 1000:
            print str(trend['query']), trend['url']
            trending_list.append(trend['query'])

%23VeteransDay http://twitter.com/search?q=%23VeteransDay
%23UFC205 http://twitter.com/search?q=%23UFC205
%23Chicago http://twitter.com/search?q=%23Chicago
%22Electoral+College%22 http://twitter.com/search?q=%22Electoral+College%22
Clemson http://twitter.com/search?q=Clemson
Auburn http://twitter.com/search?q=Auburn
%23USAvMEX http://twitter.com/search?q=%23USAvMEX
%23MannequinChallenge http://twitter.com/search?q=%23MannequinChallenge
%23GetRichIn4Words http://twitter.com/search?q=%23GetRichIn4Words
%23NationalPizzaDay http://twitter.com/search?q=%23NationalPizzaDay
%23UnitedAgainstHate http://twitter.com/search?q=%23UnitedAgainstHate


In [247]:
import twitter

In [469]:
trending_list = pd.Series(trending_list).apply(lambda x: x.encode('utf-8'))
trending_list = trending_list.apply(lambda x: re.sub("[^a-zA-Z0-9\+]", "",x))
trending_list = trending_list.apply(lambda x: x.replace('23',''))
trending_list = trending_list.apply(lambda x: x.replace('22',''))

trending_list

0            VeteransDay
1                 UFC205
2                Chicago
3      Electoral+College
4                Clemson
5                 Auburn
6                USAvMEX
7     MannequinChallenge
8        GetRichIn4Words
9       NationalPizzaDay
10     UnitedAgainstHate
dtype: object

In [248]:
twitter_keys = {
    'consumer_key':        'dhZbJCvhl79hmz6eGpsy6WRfY',
    'consumer_secret':     'TaBDp9pXOlQcxbhELuyzDFilEz5onut1hDtuOAPOYmQu15Veby',
    'access_token_key':    '729878120338063360-RuyFwFOyq9rsRk0X8FMhkbnBxeasaL0',
    'access_token_secret': 'TRuE65px8xUj76ayD2ffiv3lhabTTFHBHp1R6fa92k8oU'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)

In [317]:
class twitterminer():

    request_limit   =   200 
    api             =   False
    data            =   []
    
    twitter_keys = {
    'consumer_key':        'dhZbJCvhl79hmz6eGpsy6WRfY',
    'consumer_secret':     'TaBDp9pXOlQcxbhELuyzDFilEz5onut1hDtuOAPOYmQu15Veby',
    'access_token_key':    '729878120338063360-RuyFwFOyq9rsRk0X8FMhkbnBxeasaL0',
    'access_token_secret': 'TRuE65px8xUj76ayD2ffiv3lhabTTFHBHp1R6fa92k8oU', 
}

    
    def __init__(self,  request_limit = 200):
        
        self.request_limit = request_limit
        
        # This sets the twitter API object for use internall within the class
        self.set_api()
        
    def set_api(self):
        
        self.api = twitter.Api(
            consumer_key         =   self.twitter_keys['consumer_key'],
            consumer_secret      =   self.twitter_keys['consumer_secret'],
            access_token_key     =   self.twitter_keys['access_token_key'],
            access_token_secret  =   self.twitter_keys['access_token_secret']
        ,sleep_on_rate_limit=True)

    def mine_search_tweets(self, term, mine_rewteets=False, max_pages = 10):

        data           =  []
        last_tweet_id  =  False
        page           =  0
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetSearch(term, count=self.request_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetSearch(term, count=self.request_limit)
                
            for item in statuses:
                
                mined = {
                    'tweet_id': item.id,
                    'handle': item.user.name,
                    'retweet_count': item.retweet_count,
                    'text': item.text,
                    'mined_at': datetime.datetime.now(),
                    'created_at': item.created_at,
                }
    
               
                
                last_tweet_id =   item.id
                data.append(mined)
                
            page +=  1

        return data
    

In [470]:
miner = twitterminer(request_limit=200)
tweets_mined_dict = {}
for trend in trending_list:
    print trend
    tweets_mined_dict[trend] = miner.mine_search_tweets(term = trend, max_pages = 10)

VeteransDay
UFC205
Chicago
Electoral+College
Clemson
Auburn
USAvMEX
MannequinChallenge
GetRichIn4Words
NationalPizzaDay
UnitedAgainstHate


In [434]:
extracted_tweets_dict = {}
for mined in tweets_mined_dict:
    print mined
    extracted_tweets_dict[mined] = []
    for indiv_tweet in tweets_mined_dict[mined]:
        extracted_tweets_dict[mined].append(indiv_tweet['text'])

tweets_df_dict = {}
for name in extracted_tweets_dict:
    tweets_df_dict[name] = cloud_tweets(extracted_tweets_dict[name])

%23NationalPizzaDay
%23UFC205
%23USAvMEX
%23UnitedAgainstHate
%23GetRichIn4Words
%23MannequinChallenge
Auburn
%23VeteransDay
%23Chicago
Clemson
%22Electoral+College%22


In [445]:
from sklearn.feature_extraction.text import CountVectorizer
tweet_df_dict = {}
stops = set(stopwords.words("english")) 

def cloud_tweets(tweets):

    import re
    from nltk.corpus import stopwords


    clean_tweets = pd.Series(tweets).apply(lambda x: re.sub('https(.)*', '', x, flags=re.MULTILINE))
    #print clean_tweets[0:5]
    clean_tweets = clean_tweets.apply(lambda x: x.split(':')[1] if ":" in x else x )
    clean_tweets = clean_tweets.apply(lambda x: str(x.encode('utf-8')))
    clean_tweets = clean_tweets.apply(lambda x: re.sub("[^a-zA-Z0-9]", " ",x))
    clean_tweets = clean_tweets.apply(lambda x: x.replace('CatsIn5Words',''))
# camel case
    clean_tweets = clean_tweets.apply(lambda x: re.sub("([A-Z])"," \g<0>",x))
    clean_tweets = clean_tweets.apply(lambda x: x.lower().split())
    clean_tweets = clean_tweets.apply(lambda x: [w for w in x if not w in stops])
    clean_tweets = clean_tweets.apply(lambda x: " ".join(x))
    

    word_df = bag_words(clean_tweets)
    return word_df
def bag_words(clean_tweets):
    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 5000) 

    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings.
    train_data_features = vectorizer.fit_transform(clean_tweets)

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    train_data_features = train_data_features.toarray()

    # Take a look at the words in the vocabulary
    vocab = vectorizer.get_feature_names()

    # Sum up the counts of each vocabulary word
    dist = np.sum(train_data_features, axis=0)

    # For each, print the vocabulary word and the number of times it 
    # appears in the training set

    count_list = []
    word_list = []
    for tag, count in zip(vocab, dist):
        #print count, tag
        count_list.append(count)
        word_list.append(tag)

    import pandas as pd
    word_df = pd.DataFrame({name: word_list,'count': count_list})

    word_df = word_df.set_index('count').sort_index(ascending = False)
    
    return word_df




In [ ]:
tweets_df_dict[]

In [441]:
short_cloud = {}
new_cloud = {}
for i in tweets_df_dict:
    try:
        new_cloud[i] = tweets_df_dict[i].set_index(i).drop('xa').reset_index().sort_values('count', ascending = False)   
        short_cloud = new_cloud[i][0:20]
    except:
        new_cloud[i] = tweets_df_dict[i].reset_index().sort_values('count', ascending = False)
        short_cloud[i] = new_cloud[i][0:20]


In [443]:
short_cloud['%22Electoral+College%22']

,count,%22Electoral+College%22
0,788,electoral
1,766,college
2,203,trump
3,184,vote
4,140,clinton
5,120,sign
6,108,president
7,101,petition
8,96,hillary
9,90,popular


In [466]:
initial_column = str(short_cloud.keys()[0])
final_cloud_df = pd.DataFrame({initial_column:short_cloud[initial_column][initial_column],initial_column+'_count':short_cloud[initial_column]['count']})

for i in short_cloud:
    if i != initial_column:
        
        #print i, np.array(short_cloud[i]['word'])[0:5], pd.Series(short_cloud[i]['count'])[0:5]
        final_cloud_df[i] = short_cloud[i][i].tolist()
        final_cloud_df[i+'_count'] = short_cloud[i]['count'].tolist()
    else:
        continue

In [467]:
final_cloud_df

,%23NationalPizzaDay,%23NationalPizzaDay_count,%23UFC205,%23UFC205_count,%23USAvMEX,%23USAvMEX_count,%23Chicago,%23Chicago_count,%23GetRichIn4Words,%23GetRichIn4Words_count,...,Auburn,Auburn_count,%23VeteransDay,%23VeteransDay_count,%23UnitedAgainstHate,%23UnitedAgainstHate_count,Clemson,Clemson_count,%22Electoral+College%22,%22Electoral+College%22_count
0,pizza,1339,c205,695,av,1016,chicago,1071,get,1120,...,auburn,945,veterans,1131,hate,1130,clemson,815,electoral,788
1,day,1036,ufc205,380,wall,322,trump,969,rich,1095,...,georgia,278,day,988,united,963,pitt,227,college,766
2,national,1005,edgar,234,unity,279,providing,492,words,1048,...,game,152,thank,264,trump,365,pittsburgh,146,trump,203
3,happy,162,frankie,218,divide,274,officers,491,in4,1044,...,since,95,served,135,protest,252,top,145,vote,184
4,trump,60,khabib,182,mexico,130,protest,442,trump,91,...,13,89,trump,128,downtown,214,death,132,clinton,140
5,nationalpizzaday,57,stephens,104,la,114,thank,425,give,76,...,half,80,honor,114,000,189,valley,131,sign,120
6,get,54,garden,95,el,113,donald,424,money,71,...,beat,76,service,110,100,189,playoff,126,president,108
7,today,52,guy,91,miseleccionmx,109,spread,424,start,70,...,team,76,amp,99,wilshire,183,opponent,126,petition,101
8,like,45,team,89,por,101,real,424,clinton,66,...,today,73,today,91,notmypresident,180,since,123,hillary,96
9,week,45,vs,78,victoria,101,tower,82,foundation,56,...,win,68,country,79,tr,171,win,104,popular,90


In [ ]:
for i in short_cloud:
    if i != 'Ichido':
        #short_cloud[i].sort_values('count', ascending = False, inplace = True)
        print i, np.array(short_cloud[i]['word'])[0:5], pd.Series(short_cloud[i]['count'])[0:5]
        final_cloud_df[i] = short_cloud[i]['word'].tolist()
        final_cloud_df[i+'_count'] = short_cloud[i]['count'].tolist()
    else:
        continue